# Parametric Bootstrapping Force Peak for Unfolding

In [ ]:
include("gibbs.jl");
plt = palette(:default);

## Inputs to be specified by user

In [ ]:
NL = 24; # Number of linker regions in CDH23
NM = 80; # Number of CDH23 monomers in a bundle

## Load the Gibbs samples for $\mathrm{Ca}^{2+}$ state

In [ ]:
dfsmp = CSV.read("gibbssmp.csv",DataFrame);
SMP = [dfsmp[i,j] for i=1:nrow(dfsmp),j=1:ncol(dfsmp)];
_,nCa = size(SMP);
SMP = reshape(SMP,8,:,nCa);
ngibbs = size(SMP)[2];
println("Gibbs samples are stored in SMP which is nCfg x ngibbs x nCa array.");
println("Configs's are ordered like p010 p100 p101 p110 p000 p001 p011 p111 along first index.");

## Load the Gibbs samples for force peaks at unfolding

In [ ]:
df_μσjw = CSV.read("gibbssmps_unfoldinghypers.csv",DataFrame);
n_μσjw = ncol(df_μσjw)
M_μσjw = [df_μσjw[i,j] for i=1:nrow(df_μσjw),j=1:ncol(df_μσjw)];

## Load the Gibbs samples for unfolding extensions

In [ ]:
df_ext = CSV.read("gmm_extvsfrc.csv",DataFrame)
nclusters = nrow(df_ext)÷5
M_gmm  = [df_ext[i,j] for i=1:nrow(df_ext),j=1:ncol(df_ext)]
M_gmmw = M_gmm[1:nclusters,:]
M_gmmμ = M_gmm[nclusters+1:nclusters+2*nclusters,:]
M_gmmσ = .√(M_gmm[nclusters+2*nclusters+1:nclusters+2*nclusters+2*nclusters,:]);

## Parametric bootstrap CDH23 monomer unfolding forces

In [ ]:
nsmp = 10000;
fsmps = Matrix{Float64}(undef,nCa,nsmp); mnr = Vector{Float64}(undef,NL); 
extsmps = Matrix{Float64}(undef,nCa,nsmp); Δextsmps = Matrix{Float64}(undef,nCa,nsmp)
ps = fill(0.,8); ps[1] = 1.; P = Categorical(ps);
frc_μ = Vector{Float64}(undef,8); frc_σ = similar(frc_μ); frc_j = similar(frc_μ); frc_ws = Matrix{Float64}(undef,8,8)
typs = Vector{Int64}(undef,NL); dxndns0 = Vector{Float64}(undef,NL); dxndns = Vector{Float64}(undef,NL)
logfws_tmp = Vector{Float64}(undef,nclusters); logλs = Vector{Float64}(undef,nclusters); λs = Vector{Float64}(undef,nclusters) 
for i=1:nCa,j=1:nsmp,k=1:NL
    if k==1
        # redraw a gibbs Ca2+ distribution for this monomer
        id = rand(1:ngibbs);
        ps[:] = SMP[:,id,i];
        P.p[:] = ps;
        
        # redraw the unfolding force profiles for Ca2+ states
        id = rand(1:n_μσjw)
        frc_μ[:] = M_μσjw[1:8,id]
        frc_σ[:] = M_μσjw[9:16,id]
        frc_j[:] = M_μσjw[17:24,id]
        frc_ws[:,:] = reshape(M_μσjw[25:end,id],8,8)
    end
    
    # fill the linker region with Ca²⁺ state
    typ = rand(P); typs[k] = typ
    
    # sample an unfolding force from parametric distribution
    ## extract the mixture unfolding event profile for unfolding for this Ca2+ state
    jid = Int64(frc_j[typ]); w = frc_ws[:,jid] 
    
    ## sample the event conditional on the mixture profile
    C = Categorical(w); id = rand(C)
    μ = frc_μ[id]; σ = frc_σ[id]
    
    mnr[k] = μ+σ*randn();
    mnr[k] = mnr[k] >= 0 ? mnr[k] : 0
    
    if k==NL
        # compute the unfolding force across full monomer
        ufrc = minimum(mnr)
        fsmps[i,j] = ufrc;
        
        # sample the dxndn extensions across the full monomer
        for ℓ=1:NL
            typ = typs[ℓ]
            
            #  extract the gmm parameters for this linker
            dxndn_ws = @view M_gmmw[:,typ]
            dxndn_μs = reshape(M_gmmμ[:,typ],2,:)
            dxndn_σs = reshape(M_gmmσ[:,typ],2,:)
            
            #  compute the renormalized conditional weights in the gmm
            #  at 0 force to get starting position
            logfws_tmp[:] .= [-0.5*((0. -dxndn_μs[2,q])/dxndn_σs[2,q])^2 - log(dxndn_σs[2,q])-0.5*log(2π) for q=1:nclusters]
            logλs[:] .= [log(dxndn_ws[q])+logfws_tmp[q] for q=1:nclusters]
            #  since we take a ratio, we can shift the log by constant term which helps mitigate underflow
            λshift = maximum(logλs); logλs = logλs .- λshift
            λs[:] .= exp.(logλs)
            λs[:] .= λs./sum(λs)
            
            #  sample the 0 extension
            Cext = Categorical(λs); gmmid = rand(Cext)
            dxndns0[ℓ] = dxndn_μs[1,gmmid] + dxndn_σs[1,gmmid]*randn()
            
            #  compute the renormalized conditional weights in the gmm
            #  for this unfolding force
            #  the x-coordinate was the extension and y was the force
            logfws_tmp[:] .= [-0.5*((ufrc -dxndn_μs[2,q])/dxndn_σs[2,q])^2 - log(dxndn_σs[2,q])-0.5*log(2π) for q=1:nclusters]
            logλs[:] .= [log(dxndn_ws[q])+logfws_tmp[q] for q=1:nclusters]
            #  since we take a ratio, we can shift the log by constant term which helps mitigate underflow
            λshift = maximum(logλs); logλs = logλs .- λshift
            λs[:] .= exp.(logλs)
            λs[:] .= λs./sum(λs)
                
            # sample the extension
            Cext = Categorical(λs); gmmid = rand(Cext)
            dxndns[ℓ] = (dxndn_μs[1,gmmid] + dxndn_σs[1,gmmid]*randn())
        end
        
        # aggregate the total extension
        extsmps[i,j]  = sum(dxndns)
        Δextsmps[i,j] = sum(dxndns-dxndns0)
    end
end

### Plot tiplink unfolding forces as function of $[Ca^{2+}]$

In [ ]:
fμ = sum(fsmps,dims=2)/nsmp; 
lw = [minimum(fsmps[ℓ,:]) for ℓ=1:nCa]; up = [maximum(fsmps[ℓ,:]) for ℓ=1:nCa];

plot(1:nCa,fμ,linewidth=3,ribbon=(fμ-lw,up-fμ),fillalpha=0.1,legend=:topleft,
     xlabel = "[Ca²⁺] (μM)",ylabel="force (pN)",size=(450,300),labels="");

lw = [quantile(fsmps[ℓ,:],0.025) for ℓ=1:nCa]; up = [quantile(fsmps[ℓ,:],0.975) for ℓ=1:nCa];

plot!(1:nCa,fμ,linewidth=0,ribbon=(fμ-lw,up-fμ),fillalpha=0.2,
     xlabel = "[Ca²⁺] (μM)",ylabel="force (pN)",labels="",c=plt[1]);

In [ ]:
lw = [quantile(fsmps[ℓ,:],0.25) for ℓ=1:nCa]; up = [quantile(fsmps[ℓ,:],0.75) for ℓ=1:nCa];

plot!(1:nCa,fμ,linewidth=0,ribbon=(fμ-lw,up-fμ),fillalpha=0.35,
     xlabel = "[Ca²⁺] (μM)",ylabel="force (pN)",labels="",c=plt[1])

plot!(xtickfontsize=10,ytickfontsize=10,fontsize=12,legendfontsize=10,titlefontsize=14);

In [ ]:
p1 = plot!();
p2 = deepcopy(p1);
plot!(p2,xlims=(0,50));

In [ ]:
fμ = sum(fsmps,dims=2)/nsmp; 
lw = [minimum(fsmps[ℓ,:]) for ℓ=1:nCa]; up = [maximum(fsmps[ℓ,:]) for ℓ=1:nCa];

plot!(p2,inset=(1,bbox(0.025,0.1,0.45,0.35,:bottom,:right)));

plot!(p2[2],1:nCa,fμ,linewidth=3,ribbon=(fμ-lw,up-fμ),fillalpha=0.1,labels="");

In [ ]:
lw = [quantile(fsmps[ℓ,:],0.025) for ℓ=1:nCa]; up = [quantile(fsmps[ℓ,:],0.975) for ℓ=1:nCa];

plot!(p2[2],1:nCa,fμ,linewidth=0,ribbon=(fμ-lw,up-fμ),fillalpha=0.2,
     labels="",c=plt[1]);

In [ ]:
lw = [quantile(fsmps[ℓ,:],0.25) for ℓ=1:nCa]; up = [quantile(fsmps[ℓ,:],0.75) for ℓ=1:nCa];

plot!(p2[2],1:nCa,fμ,linewidth=0,ribbon=(fμ-lw,up-fμ),fillalpha=0.35,
     labels="",c=plt[1])

In [ ]:
savefig("predfrpt.pdf");

### Plot CDH23 monomer extensions as function of $[Ca^{2+}]$

In [ ]:
extμ = sum(extsmps,dims=2)/nsmp; 
lw = [minimum(extsmps[ℓ,:]) for ℓ=1:nCa]; up = [maximum(extsmps[ℓ,:]) for ℓ=1:nCa];

plot(1:nCa,extμ,linewidth=3,ribbon=(extμ-lw,up-extμ),fillalpha=0.1,legend=:bottomright,
     xlabel = "[Ca²⁺] (μM)",ylabel="tip link length (Å)",size=(450,300),labels="");

lw = [quantile(extsmps[ℓ,:],0.025) for ℓ=1:nCa]; up = [quantile(extsmps[ℓ,:],0.975) for ℓ=1:nCa];

plot!(1:nCa,extμ,linewidth=0,ribbon=(extμ-lw,up-extμ),fillalpha=0.2,
     xlabel = "[Ca²⁺] (μM)",ylabel="tip link length (Å)",labels="",c=plt[1]);

In [ ]:
lw = [quantile(extsmps[ℓ,:],0.25) for ℓ=1:nCa]; up = [quantile(extsmps[ℓ,:],0.75) for ℓ=1:nCa];

plot!(1:nCa,extμ,linewidth=0,ribbon=(extμ-lw,up-extμ),fillalpha=0.35,
     xlabel = "[Ca²⁺] (μM)",ylabel="tip link length (Å)",labels="",c=plt[1])

plot!(xtickfontsize=10,ytickfontsize=10,fontsize=12,legendfontsize=10,titlefontsize=14);

In [ ]:
p1 = plot!();
p2 = deepcopy(p1);
plot!(p2,xlims=(0,50));

In [ ]:
extμ = sum(extsmps,dims=2)/nsmp; 
lw = [minimum(extsmps[ℓ,:]) for ℓ=1:nCa]; up = [maximum(extsmps[ℓ,:]) for ℓ=1:nCa];

plot!(p2,inset=(1,bbox(0.12,0.125,0.45,0.35,:bottom,:left)));

plot!(p2[2],1:nCa,extμ,linewidth=3,ribbon=(extμ-lw,up-extμ),fillalpha=0.1,labels="");

In [ ]:
lw = [quantile(extsmps[ℓ,:],0.025) for ℓ=1:nCa]; up = [quantile(extsmps[ℓ,:],0.975) for ℓ=1:nCa];

plot!(p2[2],1:nCa,extμ,linewidth=0,ribbon=(extμ-lw,up-extμ),fillalpha=0.2,
     labels="",c=plt[1]);

In [ ]:
lw = [quantile(extsmps[ℓ,:],0.25) for ℓ=1:nCa]; up = [quantile(extsmps[ℓ,:],0.75) for ℓ=1:nCa];

plot!(p2[2],1:nCa,extμ,linewidth=0,ribbon=(extμ-lw,up-extμ),fillalpha=0.35,
     labels="",c=plt[1])

In [ ]:
Δextμ = sum(Δextsmps,dims=2)/nsmp; 
lw = [minimum(Δextsmps[ℓ,:]) for ℓ=1:nCa]; up = [maximum(Δextsmps[ℓ,:]) for ℓ=1:nCa];

plot(1:nCa,Δextμ,linewidth=3,ribbon=(Δextμ-lw,up-Δextμ),fillalpha=0.1,legend=:bottomright,
     xlabel = "[Ca²⁺] (μM)",ylabel="tip link Δlength (Å)",size=(450,300),labels="");

lw = [quantile(Δextsmps[ℓ,:],0.025) for ℓ=1:nCa]; up = [quantile(Δextsmps[ℓ,:],0.975) for ℓ=1:nCa];

plot!(1:nCa,Δextμ,linewidth=0,ribbon=(Δextμ-lw,up-Δextμ),fillalpha=0.2,
     xlabel = "[Ca²⁺] (μM)",ylabel="tip link Δlength (Å)",labels="",c=plt[1]);

In [ ]:
lw = [quantile(Δextsmps[ℓ,:],0.25) for ℓ=1:nCa]; up = [quantile(Δextsmps[ℓ,:],0.75) for ℓ=1:nCa];

plot!(1:nCa,Δextμ,linewidth=0,ribbon=(Δextμ-lw,up-Δextμ),fillalpha=0.35,
     xlabel = "[Ca²⁺] (μM)",ylabel="tip link Δlength (Å)",labels="",c=plt[1])

plot!(xtickfontsize=10,ytickfontsize=10,fontsize=12,legendfontsize=10,titlefontsize=14);

In [ ]:
p3 = plot!();
p4 = deepcopy(p3);
plot!(p4,xlims=(0,50));

In [ ]:
Δextμ = sum(Δextsmps,dims=2)/nsmp; 
lw = [minimum(Δextsmps[ℓ,:]) for ℓ=1:nCa]; up = [maximum(Δextsmps[ℓ,:]) for ℓ=1:nCa];

plot!(p4,inset=(1,bbox(0.05,0.1,0.45,0.35,:top,:right)));

plot!(p4[2],1:nCa,Δextμ,linewidth=3,ribbon=(Δextμ-lw,up-Δextμ),fillalpha=0.1,labels="");

In [ ]:
lw = [quantile(Δextsmps[ℓ,:],0.025) for ℓ=1:nCa]; up = [quantile(Δextsmps[ℓ,:],0.975) for ℓ=1:nCa];

plot!(p4[2],1:nCa,Δextμ,linewidth=0,ribbon=(Δextμ-lw,up-Δextμ),fillalpha=0.2,
     labels="",c=plt[1]);

In [ ]:
lw = [quantile(Δextsmps[ℓ,:],0.25) for ℓ=1:nCa]; up = [quantile(Δextsmps[ℓ,:],0.75) for ℓ=1:nCa];

plot!(p4[2],1:nCa,Δextμ,linewidth=0,ribbon=(Δextμ-lw,up-Δextμ),fillalpha=0.35,
     labels="",c=plt[1])

In [ ]:
plot(p2,p4,size=(900,300),margin=4mm)
savefig("predextrpt.pdf");

## Parametric bootstrap bundle ruptures

In [ ]:
nsmp = 10000;
fsmps = Matrix{Float64}(undef,nCa,nsmp); 
mnr = Vector{Float64}(undef,NL); bndl = Vector{Float64}(undef,NM);
ps = fill(0.,8); ps[1] = 1.; P = Categorical(ps);
frc_μ = Vector{Float64}(undef,8); frc_σ = similar(frc_μ)

prg = 0.; δprg = 0.025
for i=1:nCa,j=1:nsmp,k=1:NM,ℓ=1:NL
    if k==1
        # redraw a gibbs Ca2+ distribution for this bundle
        id = rand(1:ngibbs);
        ps[:] = SMP[:,id,i];
        P.p[:] = ps;
        
        # redraw the unfolding force profiles for Ca2+ states
        id = rand(1:n_μσjw)
        frc_μ[:] = M_μσjw[1:8,id]
        frc_σ[:] = M_μσjw[9:16,id]
        frc_j[:] = M_μσjw[17:24,id]
        frc_ws[:,:] = reshape(M_μσjw[25:end,id],8,8)
    end
    
    # fill the linker region with Ca²⁺ state
    typ = rand(P);
    
    # sample an unfolding force from parametric distribution
    ## Extract the mixture unfolding event profile for unfolding for this Ca2+ state
    jid = Int64(frc_j[typ]); w = frc_ws[:,jid] 
    
    ## Sample the event conditional on the mixture profile
    C = Categorical(w); id = rand(C)
    μ = frc_μ[id]; σ = frc_σ[id]
    
    mnr[ℓ] = μ+σ*randn();
    
    if ℓ==NL
        # compute the unfolding force across full monomer
        bndl[k] = minimum(mnr);
    end
    
    if k==NM
        # compute 50% of tiplinks are unfolded force for the bundle
        fsmps[i,j] = quantile(bndl,0.5);
    end
    
    while i/nCa >= prg+δprg
        prg += δprg
        println("progress through samples $(prg)...")
    end
end

### Plot 50% bundle rupture forces as function of $[Ca^{2+}]$

In [ ]:
fμ = sum(fsmps,dims=2)/nsmp; 
lw = [minimum(fsmps[ℓ,:]) for ℓ=1:nCa]; up = [maximum(fsmps[ℓ,:]) for ℓ=1:nCa];

plot(1:nCa,fμ,linewidth=3,ribbon=(fμ-lw,up-fμ),fillalpha=0.1,legend=:topleft,
     xlabel = "[Ca²⁺] (μM)",ylabel="force (pN)",size=(450,300),labels="");

In [ ]:
lw = [quantile(fsmps[ℓ,:],0.025) for ℓ=1:nCa]; up = [quantile(fsmps[ℓ,:],0.975) for ℓ=1:nCa];

plot!(1:nCa,fμ,linewidth=0,ribbon=(fμ-lw,up-fμ),fillalpha=0.2,
     xlabel = "[Ca²⁺] (μM)",ylabel="force (pN)",labels="",c=plt[1]);

In [ ]:
lw = [quantile(fsmps[ℓ,:],0.25) for ℓ=1:nCa]; up = [quantile(fsmps[ℓ,:],0.75) for ℓ=1:nCa];

plot!(1:nCa,fμ,linewidth=0,ribbon=(fμ-lw,up-fμ),fillalpha=0.35,
     xlabel = "[Ca²⁺] (μM)",ylabel="force (pN)",labels="",c=plt[1])

plot!(xtickfontsize=10,ytickfontsize=10,fontsize=12,legendfontsize=10,titlefontsize=14);

In [ ]:
p1 = plot!();
p2 = deepcopy(p1);
plot!(p2,xlims=(0,50));

In [ ]:
fμ = sum(fsmps,dims=2)/nsmp; 
lw = [minimum(fsmps[ℓ,:]) for ℓ=1:nCa]; up = [maximum(fsmps[ℓ,:]) for ℓ=1:nCa];

plot!(p2,inset=(1,bbox(0.025,0.1,0.45,0.35,:bottom,:right)));

plot!(p2[2],1:nCa,fμ,linewidth=3,ribbon=(fμ-lw,up-fμ),fillalpha=0.1,labels="");

In [ ]:
lw = [quantile(fsmps[ℓ,:],0.025) for ℓ=1:nCa]; up = [quantile(fsmps[ℓ,:],0.975) for ℓ=1:nCa];

plot!(p2[2],1:nCa,fμ,linewidth=0,ribbon=(fμ-lw,up-fμ),fillalpha=0.2,
     labels="",c=plt[1]);

In [ ]:
lw = [quantile(fsmps[ℓ,:],0.25) for ℓ=1:nCa]; up = [quantile(fsmps[ℓ,:],0.75) for ℓ=1:nCa];

plot!(p2[2],1:nCa,fμ,linewidth=0,ribbon=(fμ-lw,up-fμ),fillalpha=0.35,
     labels="",c=plt[1])

In [ ]:
savefig("predbrpt.pdf");
CSV.write("bndlrptsmps.csv",DataFrame(fsmps,:auto));